# Building custom components

In NVFlare, the [`FLComponent`](https://nvflare.readthedocs.io/en/2.5.2/apidocs/nvflare.apis.fl_component.html#nvflare.apis.fl_component.FLComponent) is the base class of all components, including controllers, executors, responders, filters, aggregators and many others (see the [here](https://nvflare.readthedocs.io/en/2.5.2/programming_guide/fl_component.html) for more details). `FLComponent`s have the capability to handle and fire events and contain various methods for logging.

In this notebook, we'll explore how to implement a very basic custom `Controller` and `Executor` in NVFlare:
- a `Controller` is a server-side component responsible for managing job execution and orchestrating tasks across clients
- an `Executor` is a client-side component that processes tasks received from the controller and executes them accordingly

For this first tutorial we won't go into the details of tasks and execution. Instead, we'll start by creating dummy versions of these components and then add some very basic functionality to them to better understand the flow of execution and the roles of the different components involved.

## The Controller
We'll start by creating a `DummyController` class that doesn't perform any specific tasks but serves as a template for understanding how controllers work in NVFlare.

As a subclass of `Controller`, our `DummyController` must implement three methods:

- `control_flow`: defining the main control flow of the controller. It receives an `abort_signal: Signal` and an `fl_ctx: FLContext`. A [`Signal`](https://nvflare.readthedocs.io/en/2.5.2/apidocs/nvflare.apis.signal.html#nvflare.apis.signal.Signal) is an object that provides a mechanism to signal events like abortion. Controllers and executors can check this signal to determine if they should stop execution gracefully. More details on handling abort signals [here](https://nvflare.readthedocs.io/en/2.5.2/best_practices.html#respect-the-abort-signal). An [`FLContext`](https://nvflare.readthedocs.io/en/2.5.2/apidocs/nvflare.apis.fl_context.html#nvflare.apis.fl_context.FLContext) object carries all the execution context, which includes information about the current execution environment, such as run number, job ID, and other configurations. It's passed to many methods to provide context. More details can be found [here](https://nvflare.readthedocs.io/en/2.5.2/programming_guide/fl_context.html). We'll talk more about it in the next notebooks.
- `start_controller`: called once before the `control_flow` method. It's used to perform any setup tasks or initialize resources.
- `stop_controller`: called once after the `control_flow` method. It's used to clean up resources or perform any finalization tasks.

For the moment, we'll leave these methods empty. We'll add some functionality to them later in the notebook.

```python
from nvflare.apis.fl_context import FLContext
from nvflare.apis.impl.controller import Controller
from nvflare.apis.signal import Signal

class DummyController(Controller):

    def control_flow(self, abort_signal: Signal, fl_ctx: FLContext):
        pass

    def start_controller(self, fl_ctx: FLContext):
        pass

    def stop_controller(self, fl_ctx: FLContext):
        pass
```

Let's try to instantiate the controller and run it via the NVFlare simulator. As we've seen in previous chapters, objects sent to server and clients need to be importable by the simulator. Since we are in a notebook, we can't import the controller directly, so let's put it in a separate file `modules.py` and import it from there - for convenience, we have already included all the custom components we'll use in this chapter in `modules.py`, but feel free to delete the file and recreate it from scratch while we move forward.

In [1]:
from nvflare.job_config.api import FedJob
from modules import DummyController

# Create job
job = FedJob(name="dummy_job")

# send controller to server
controller = DummyController()
job.to_server(controller)

# Run job via the NVFlare simulator
job.simulator_run("./tmp/")

2025-02-05 15:58:34,037 - SimulatorRunner - ERROR - Simulator setup error: StopIteration: 
2025-02-05 15:58:34,038 - root - ERROR - Traceback (most recent call last):
  File "/Users/ffarina/Documents/code/NVFlare/nvflare/private/fed/app/simulator/simulator_runner.py", line 212, in setup
    data_bytes, job_name, meta = self.validate_job_data()
                                 ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ffarina/Documents/code/NVFlare/nvflare/private/fed/app/simulator/simulator_runner.py", line 339, in validate_job_data
    valid, error, meta = job_validator.validate(job_name, data_bytes)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ffarina/Documents/code/NVFlare/nvflare/private/fed/server/job_meta_validator.py", line 66, in validate
    meta = self._validate_zf(job_name, zf)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ffarina/Documents/code/NVFlare/nvflare/private/fed/server/job_meta_validator.py", line 46, in _validate_

Now, as you can see, trying to run the previous job results in an error. This is because we have only created a server running our `DummyController` but ut we haven't sent any executors to the clients. 

To fix that, let's move on to the next section to explore `Executor`s and create a custom one.

## The Executor

When subclassing an `Executor` the main method that must be implmented is the `execute` method, which defines how the `Executor` processes a task received from the `Controller`. It receives the following arguments in addition to an `abort_signal` and `fl_ctx`:
- `task_name`: the name of the task to be executed
- `shareable`: a [`Shareable`](https://nvflare.readthedocs.io/en/2.5.2/apidocs/nvflare.apis.shareable.html#nvflare.apis.shareable.Shareable) object containing the task data. We'll talk more about it in the next notebook but for the moment just note that this is the data structure used for communication between the server and clients. It wraps the data exchanged and can include metadata or headers. More details on `Shareable`s [here](https://nvflare.readthedocs.io/en/2.5.2/programming_guide/shareable.html).

Let's create a `DummyExecutor` class that doesn't perform any specific tasks but serves as a template for understanding how executors work in NVFlare.

```python
class DummyExecutor(Executor):

    def execute(
        self,
        task_name: str,
        shareable: Shareable,
        fl_ctx: FLContext,
        abort_signal: Signal,
    ):
        pass
```

Let's now add 3 clients, assign our `DummyExecutor` to each of them and run our job.

In [2]:
from modules import DummyExecutor

# Create job
job = FedJob(name="dummy_job")

# send controller to server
controller = DummyController()
job.to_server(controller)

# send executor to clients
num_clients = 3
for i in range(num_clients):
    executor = DummyExecutor()
    job.to(executor, f"site-{i}")

# Run job via the NVFlare simulator
job.simulator_run("./tmp/")

2025-02-05 15:58:38,237 - SimulatorRunner - INFO - Create the Simulator Server.
2025-02-05 15:58:38,238 - CoreCell - INFO - server: creating listener on tcp://0:52379
2025-02-05 15:58:38,265 - CoreCell - INFO - server: created backbone external listener for tcp://0:52379
2025-02-05 15:58:38,265 - ConnectorManager - INFO - 44582: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2025-02-05 15:58:38,266 - conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:1029] is starting
2025-02-05 15:58:38,771 - CoreCell - INFO - server: created backbone internal listener for tcp://localhost:1029
2025-02-05 15:58:38,773 - conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:52379] is starting
2025-02-05 15:58:38,776 - AuxRunner - INFO - registered aux handler for topic ObjectStreamer.Request
2025-02-05 15:58:38,776 - AuxRunner - INFO - registered aux handler for topic ObjectStreamer.Abort
2025-02-05 15:58:38,777 - Cell - INFO - Register blob CB for channel='aux_co

When you run the above code, you'll notice that the number of `Total clients` in the system increases from 1 to 3 as the clients start. However, since both the controller and executors are not performing any specific operations (their methods are empty), the job completes without any significant actions.

## Logging info

To explore the actual flow of execution, let's add some very basic logging to our `DummyController` and `DummyExecutor`. 

We want both our controller and executors to log when they start and stop but we'll do that in two different ways. 

For the controller, we'll do that in the `start_controller` and `stop_controller` methods.
For the executors, we'll do that by overriding their `handle_event` method, which as the name suggests, is used to handle different events. Notice that NVFlare uses an event-driven architecture where components can fire and handle events to coordinate actions. By checking for `EventType.START_RUN` or `EventType.END_RUN` events we can log messages when the executor starts and stops.

We can use the `log_info` method which is available to any subclass of `FLComponent` (which both `Controller` and `Executor` inherit from).

```python
class DummyLoggingController(Controller):
    def control_flow(self, abort_signal: Signal, fl_ctx: FLContext):
        pass

    def start_controller(self, fl_ctx: FLContext):
        self.log_info(fl_ctx, "Starting the controller...")

    def stop_controller(self, fl_ctx: FLContext):
        self.log_info(fl_ctx, "Stopping the controller...")


class DummyLoggingExecutor(Executor):
    def execute(
        self,
        task_name: str,
        shareable: Shareable,
        fl_ctx: FLContext,
        abort_signal: Signal,
    ):
        pass

    def handle_event(self, event_type, fl_ctx):
        if event_type == EventType.START_RUN:
            self.log_info(fl_ctx, "Starting the executor...")
        elif event_type == EventType.END_RUN:
            self.log_info(fl_ctx, "Stopping the executor...")
```

In [3]:
from modules import DummyLoggingController, DummyLoggingExecutor

# Create job
job = FedJob(name="dummy_job")

# send controller to server
controller = DummyLoggingController()
job.to_server(controller)

# send executor to clients
num_clients = 3
for i in range(num_clients):
    executor = DummyLoggingExecutor()
    job.to(executor, f"site-{i}")

# Run job via the NVFlare simulator
job.simulator_run("./tmp/")

2025-02-05 15:58:49,037 - SimulatorRunner - INFO - Create the Simulator Server.
2025-02-05 15:58:49,038 - CoreCell - INFO - server: creating listener on tcp://0:52529
2025-02-05 15:58:49,062 - CoreCell - INFO - server: created backbone external listener for tcp://0:52529
2025-02-05 15:58:49,062 - ConnectorManager - INFO - 44643: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2025-02-05 15:58:49,063 - conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:3934] is starting
2025-02-05 15:58:49,564 - CoreCell - INFO - server: created backbone internal listener for tcp://localhost:3934
2025-02-05 15:58:49,564 - conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:52529] is starting
2025-02-05 15:58:49,565 - AuxRunner - INFO - registered aux handler for topic ObjectStreamer.Request
2025-02-05 15:58:49,565 - AuxRunner - INFO - registered aux handler for topic ObjectStreamer.Abort
2025-02-05 15:58:49,565 - Cell - INFO - Register blob CB for channel='aux_co

Now, if you look at the output of the cell above and look for the logs of the `DummyLoggingController` and `DummyLoggingExecutor`s, you should see that they are logging the start and stop events. You should also see that the controller is logging the start and stop events before and after the executors are logging theirs. We'll see more about that in the next section.

## What's next
In the next chapter, we'll explore how to make the controller and the executors communicate with each other.